使用**nvidia-smi**命令来查看显卡信息

在PyTorch中，每个数组都有一个设备（device）， 我们通常将其称为上下文（context）。 默认情况下，所有变量和相关的计算都分配给CPU。 有时上下文可能是GPU。 当我们跨多个服务器部署作业时，事情会变得更加棘手。 通过智能地将数组分配给上下文， 我们可以最大限度地减少在设备之间传输数据的时间。 例如，当在带有GPU的服务器上训练神经网络时， 我们通常希望模型的参数在GPU上。

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

在PyTorch中，CPU和GPU可以用torch.device('cpu') 和torch.device('cuda')表示。 应该注意的是，cpu设备意味着所有物理CPU和内存， 这意味着PyTorch的计算将尝试使用所有CPU核心。 然而，gpu设备只代表一个卡和相应的显存。 如果有多个GPU，我们使用torch.device(f'cuda:{i}') 来表示第i块GPU（i从0开始）。 另外，cuda:0和cuda是等价的。

查询可用gpu的数量

In [4]:
torch.cuda.device_count()

1

定义了两个方便的函数， 这两个函数允许我们在不存在所需所有GPU的情况下运行代码

In [5]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

查询张量所在的设备。 默认情况下，张量是在CPU上创建的

In [6]:
x = torch.tensor([1,2,3])
x.device

device(type='cpu')

无论何时我们要对多个项进行操作， 它们都必须在同一个设备上。 例如，如果我们对两个张量求和， 我们需要确保两个张量都位于同一个设备上， 否则框架将不知道在哪里存储结果，甚至不知道在哪里执行计算。

有几种方法可以在GPU上存储张量。 例如，我们可以在创建张量时指定存储设备。接 下来，我们在第一个gpu上创建张量变量X。 在GPU上创建的张量只消耗这个GPU的显存。 我们可以使用nvidia-smi命令查看显存使用情况。 一般来说，我们需要确保不创建超过GPU显存限制的数据。

In [7]:
x = torch.ones(2, 3, device=try_gpu())
x

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

如果我们要计算X + Y，我们需要决定在哪里执行这个操作。我们可以将X传输到第二个GPU并在那里执行操作。 不要简单地X加上Y，因为这会导致异常， 运行时引擎不知道该怎么做：它在同一设备上找不到数据会导致失败。 由于Y位于第二个GPU上，所以我们需要将X移到那里， 然后才能执行相加运算。

类似地，神经网络模型可以指定设备。 下面的代码将模型参数放在GPU上

In [9]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [11]:
net(x)

tensor([[-0.3044],
        [-0.3044]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [12]:
net[0].weight.data.device

device(type='cuda', index=0)